In [ ]:
# A script which takes a folder path as input. The script iterates through the images inside the folder
# and outputs the corresponding classification scores in a csv file.
# Can be used for every model saved as .pth without knowing the model.
# transformations need to be the same as on validation data, needs to be adapted for the model.
# acceptable_formats can be changed to the image formats the model accepts.

# Requirement: Save the model with class labels:
# class_labels = train_dataset.classes
# import pickle
# Save the model and its classes
# with open('model.pkl', 'wb') as f:
#    pickle.dump((model, class_labels), f)


import torch
import csv
import os
from PIL import Image
import pickle

# Provide the folder path
folder_path = "/content/image_folders"

# Load the saved model and class labels
with open('model.pkl', 'rb') as f:
    model, class_labels = pickle.load(f)

# Define the transformations
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(48),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def predict_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    output = model(image)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    return probabilities.tolist()[0]

def predict_folder(folder_path):
    results = []
    acceptable_formats = ['.jpg', '.png', '.jpeg', '.tiff', '.bmp']
    for filename in os.listdir(folder_path):
        _, ext = os.path.splitext(filename)
        if ext.lower() in acceptable_formats:
            filepath = os.path.join(folder_path, filename)
            label = predict_image(filepath)
            results.append((filepath, *label))
    return results

def write_to_csv(results, output_path):
    with open(output_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["filepath", *class_labels])
        writer.writerows(results)

# The Path to the csv file for the corresponding classification scores
output_path = "prediction_scores.csv"

results = predict_folder(folder_path)
rounded_results = [(row[0], *[round(float(x), 2) for x in row[1:]]) for row in results]
write_to_csv(rounded_results, output_path)